In [1]:
import os
import h5py
import fitsio
import numpy as np
from tqdm import tqdm
import astropy.table as aTable

In [6]:
from desitarget.sv3.sv3_targetmask import bgs_mask as sv3_bgs_mask

# read in target catalog

In [4]:
ff = fitsio.read('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/BGS_ANY_full.dat.fits')

In [7]:
# select only BGS targets
is_bgs_bright = (
    (ff['COADD_FIBERSTATUS'] == 0) & 
    ((ff['SV3_BGS_TARGET'] & sv3_bgs_mask['BGS_BRIGHT']) != 0) & 
    (ff['SPECTYPE'] == 'GALAXY') & 
    (ff['Z_HP'] > 0.) & (ff['Z_HP'] < 0.6) & 
    (ff['ZWARN'] == 0) & 
    (ff['DELTACHI2'] > 40.) & 
    (ff['ZERR'] < 0.0005 * (1 + ff['Z_HP'])))

is_bgs_faint = (
    (ff['COADD_FIBERSTATUS'] == 0) & 
    ((ff['SV3_BGS_TARGET'] & sv3_bgs_mask['BGS_FAINT']) != 0) & 
    (ff['SPECTYPE'] == 'GALAXY') & 
    (ff['Z_HP'] > 0.) & (ff['Z_HP'] < 0.6) & 
    (ff['ZWARN'] == 0) & 
    (ff['DELTACHI2'] > 40.) & 
    (ff['ZERR'] < 0.0005 * (1 + ff['Z_HP'])))

print('%i BGS BRIGHT z-success' % np.sum(is_bgs_bright))
print('%i BGS FAINT z-success' % np.sum(is_bgs_faint))

143074 BGS BRIGHT z-success
96771 BGS FAINT z-success


# read in VAGC

In [2]:
fvagc = '/global/cfs/cdirs/desi/users/chahah/provabgs/svda/BGS_ANY_full.provabgs.hdf5'
vagc = aTable.Table.read(fvagc)

In [3]:
vagc[:5]

TARGETID,RA,DEC,Z_HP,ZERR,TSNR2_BGS,MAG_G,MAG_R,MAG_Z,MAG_W1,FIBMAG_R,HPIX_64,"provabgs_mcmc [100,13]",provabgs_theta_bf [13],provabgs_z_max,provabgs_logMstar_bf,provabgs_logMstar [100]
int64,float64,float64,float64,float64,float32,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64
39633390185481181,191.17913199949842,60.25733587654764,0.2004070714061304,nan,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0 .. -999.0,-999.0 .. -999.0,-999.0,-999.0,-999.0 .. -999.0
39633390164510024,188.6998080571996,60.21175772175424,0.2004070714061304,nan,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0 .. -999.0,-999.0 .. -999.0,-999.0,-999.0,-999.0 .. -999.0
39633390185480955,191.14955067198017,60.34047183206205,0.2004070714061304,nan,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0 .. -999.0,-999.0 .. -999.0,-999.0,-999.0,-999.0 .. -999.0
39633390185480938,191.14757099005186,60.345815959235,0.2819459922463374,2.191690764397e-05,1361.7778,20.420939956138607,19.440977833152942,18.807486906915823,18.725561063547207,21.083244145949642,11226,11.091623656909688 .. 0.3684622521221414,11.14249424405227 .. 0.3651476796540595,0.2891440449335749,10.894820213317871,10.848259925842285 .. 10.952618598937988
39633416324383107,187.05117249536465,62.614542270156655,0.2703579310142208,0.00018002802349298926,1218.0295,20.876585604498494,19.373871860075226,18.567397256862876,18.32574859422363,20.212708263179056,11251,11.127879812361321 .. 0.6691898790548064,11.16028636319183 .. 0.6710591095679554,0.2822033708476234,10.909546852111816,10.877055168151855 .. 10.874772071838379


In [8]:
has_va = (vagc['provabgs_z_max'] != -999.)

In [11]:
print('%i of %i BGS BRIGHT has VAGC' % (np.sum(is_bgs_bright & has_va), np.sum(is_bgs_bright)))
print('%i of %i BGS FAINT has VAGC' % (np.sum(is_bgs_faint & has_va), np.sum(is_bgs_faint)))

143017 of 143074 BGS BRIGHT has VAGC
95499 of 96771 BGS FAINT has VAGC


In [12]:
print('%i of %i BGS BRIGHT does not have VAGC' % (np.sum(is_bgs_bright & ~has_va), np.sum(is_bgs_bright)))
print('%i of %i BGS FAINT does not have VAGC' % (np.sum(is_bgs_faint & ~has_va), np.sum(is_bgs_faint)))

57 of 143074 BGS BRIGHT does not have VAGC
1272 of 96771 BGS FAINT does not have VAGC
